# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [9]:
import numpy as np
import pickle

In [10]:
class Node:
    def __init__(self, density=0):
        self.density = density
        self.subclass_id = -1
    def __repr__(self):
        return str(self.density)

class Graph:
    def __init__(self):
        self.nodes = {}
        self.neighbors = {}
    def __repr__(self):
        representation = "{\n"
        for i, node in enumerate(self.nodes):
            representation += f"{i:<3}:\t {node}\n"
        representation += "}\n"
        return representation

In [13]:
# with open(r"src/suspended_undirected_graph", "rb") as file:
#     g = pickle.load(file)
# g
# with open(r"src/suspended_undirected_graph", "wb") as file:
#     pickle.dump(g, file)

with open(r"src/suspended_undirected_graph_enchanced", "rb") as file:
    g = pickle.load(file)
# with open(r"src/suspended_undirected_graph", "wb") as file:
#     pickle.dump(g, file)

In [14]:
def find_neighbors_local_maxes(g, node_id):
    apexes = set()
    visited = {node_id}
    
    queue = []
    node_density = g.nodes[node_id].density
    for neighbor_id in g.neighbors.get(node_id, set()) - visited:
        if g.nodes[neighbor_id].density > node_density:
            queue.append(neighbor_id)
        visited.add(neighbor_id)
    
    for vertex in queue:
#         print(vertex)
        is_local_max = True
        vertex_density = g.nodes[vertex].density
        for neighbor_id in g.neighbors[vertex]:
            if g.nodes[neighbor_id].density > vertex_density:
                if neighbor_id not in visited:
                    queue.append(neighbor_id)
                is_local_max = False
            visited.add(neighbor_id)
        if is_local_max:
            apexes.add(vertex)
        visited.add(vertex)
    if not apexes:
        return {node_id}
    return apexes

In [15]:
def max_apex_in_class(g, start_node_id: int):
    max_apex_id = start_node_id
    visited = {start_node_id}
    queue = list(g.neighbors.get(start_node_id, set()) - visited)
    for vertex in queue.copy():
        if g.nodes[max_apex_id].density < g.nodes[vertex].density:
            max_apex_id = vertex
        visited.add(vertex)
        queue.remove(vertex)
        queue.extend([
            node for node in g.neighbors[vertex] - visited
            if node not in visited
        ])
    return max_apex_id, visited

In [16]:
%%timeit

d = {}
for i in range(34):
    d[i+1] = find_neighbors_local_maxes(g, i+1)

1000 loops, best of 3: 339 µs per loop


In [17]:
d = {}
for i in range(34):
    d[i+1] = find_neighbors_local_maxes(g, i+1)
#     print(f"{i+1:<6}{find_neighbors_local_maxes(g, i+1)}")
print(d == check)

NameError: name 'check' is not defined

In [18]:
check = {1: {1},
 2: {2},
 3: {3},
 4: {4},
 5: {5},
 6: {6},
 7: {7},
 8: {8},
 9: {9},
 10: {10},
 11: {3, 4, 5},
 12: {1, 2, 5, 6},
 13: {1, 2, 5, 6},
 14: {1, 2, 5, 6},
 15: {1, 5, 6},
 16: {1, 2, 5, 6},
 17: {1, 6},
 18: {6},
 19: {6},
 20: {6},
 21: {6},
 22: {6},
 23: {6, 7},
 24: {7},
 25: {7},
 26: {1, 2, 5, 6, 7, 8, 9},
 27: {1, 2, 6, 7, 8, 9},
 28: {1, 2, 6, 8, 9},
 29: {8, 9},
 30: {2, 6, 8, 9},
 31: {2, 6, 8, 9, 10},
 32: {9, 10},
 33: {2, 9, 10},
 34: {2}}

In [21]:
s, l = max_apex_in_class(g, 26)
print(s)
print(l)

1
{1, 7, 8, 13, 25, 26, 29, 30}
